# LSTM Classification with CR Dataset
<hr>

We will build a text classification model using LSTM model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


sentence  label  split
0     weaknesses are minor the feel and layout of th...      0  train
1     many of our disney movies do n 't play on this...      0  train
2     player has a problem with dual layer dvd 's su...      0  train
3     i know the saying is you get what you pay for ...      0  train
4                      will never purchase apex again .      0  train
...                                                 ...    ...    ...
3770  so far , the anti spam feature seems to be ver...      1  train
3771  i downloaded a trial version of computer assoc...      1  train
3772  i did not have any of the installation problem...      1  train
3773  their products have been great and have saved ...      1  train
3774                                                         1  train

[3775 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          1368   1368
1          2407   2407

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

"weaknesses are minor the feel and layout of the remote control are only so so . it does n 't show the complete file names of mp3s with really long names . you must cycle through every zoom setting ( 2x , 3x , 4x , 1 2x , etc . ) before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly ."

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  will never purchase apex again .
Into a sequence of int: [72, 194, 285, 207, 286]
Into a padded sequence: [ 72 194 285 207 286   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
and 3
i 4
it 5
to 6
a 7
is 8
of 9
this 10
5336


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## LSTM Model

In [15]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [16]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 585,825
Trainable params: 585,825
Non-trainable params: 0
_________________________________________________________________


In [17]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [18]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
107/107 [==============================] - 78s 470ms/step - loss: 0.6128 - accuracy: 0.6679 - val_loss: 0.4231 - val_accuracy: 0.7884
Epoch 2/15
107/107 [==============================] - 44s 413ms/step - loss: 0.2795 - accuracy: 0.8922 - val_loss: 0.4191 - val_accuracy: 0.7910
Epoch 3/15
107/107 [==============================] - 48s 445ms/step - loss: 0.1241 - accuracy: 0.9616 - val_loss: 0.5523 - val_accuracy: 0.7937
Epoch 4/15
107/107 [==============================] - 42s 390ms/step - loss: 0.0839 - accuracy: 0.9714 - val_loss: 0.6551 - val_accuracy: 0.7857
Epoch 5/15
107/107 [==============================] - 43s 403ms/step - loss: 0.0420 - accuracy: 0.9861 - val_loss: 0.8798 - val_accuracy: 0.7857
Epoch 6/15
107/107 [==============================] - 40s 373ms/step - loss: 0.0245 - accuracy: 0.9918 - val_loss: 1.0067 - val_accuracy: 0.7804
Epoch 7/15
107/107 [==============================] - 42s 391ms/step - loss: 0.0146 - accuracy: 0.9959 - val_loss: 1.

107/107 [==============================] - 66s 615ms/step - loss: 0.0062 - accuracy: 0.9986 - val_loss: 1.3347 - val_accuracy: 0.7745
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 78.51458787918091
Training 8: 
Epoch 1/15
107/107 [==============================] - 145s 1s/step - loss: 0.6098 - accuracy: 0.6739 - val_loss: 0.4519 - val_accuracy: 0.7745
Epoch 2/15
107/107 [==============================] - 80s 749ms/step - loss: 0.2559 - accuracy: 0.8902 - val_loss: 0.4349 - val_accuracy: 0.7692
Epoch 3/15
107/107 [==============================] - 54s 505ms/step - loss: 0.1190 - accuracy: 0.9588 - val_loss: 0.5987 - val_accuracy: 0.7745
Epoch 4/15
107/107 [==============================] - 58s 540ms/step - loss: 0.0554 - accuracy: 0.9811 - val_loss: 0.7114 - val_accuracy: 0.7745
Epoch 5/15
107/107 [==============================] - 62s 580ms/step - loss: 0.0458 - accuracy: 0.9874 - val_loss: 0.6752 - val_accuracy: 0.7692
Epoch 6/15
10

## Summary

In [19]:
record

acc1      acc2       acc3       acc4       acc5       acc6       acc7  \
0  79.365081  79.62963  82.539684  80.158728  81.481481  80.106103  78.514588   

        acc8       acc9      acc10        AVG  
0  77.453583  78.514588  79.310346  79.707381

In [20]:
report = record
report = report.to_excel('LSTM_CR_v2.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [21]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [22]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [23]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [24]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 5046 words present from 5336 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [25]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [26]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.78736068,  0.6381698 ,  0.56861984, ..., -0.53932057,
        -0.47350975, -0.08038928],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## LSTM Model

In [27]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [28]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 129       
Total params: 585,825
Trainable params: 285,825
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [29]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [30]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/15
107/107 [==============================] - 75s 465ms/step - loss: 0.6287 - accuracy: 0.6368 - val_loss: 0.5047 - val_accuracy: 0.7302
Epoch 2/15
107/107 [==============================] - 34s 314ms/step - loss: 0.4712 - accuracy: 0.7644 - val_loss: 0.4437 - val_accuracy: 0.7751
Epoch 3/15
107/107 [==============================] - 30s 276ms/step - loss: 0.4083 - accuracy: 0.8156 - val_loss: 0.4907 - val_accuracy: 0.7407
Epoch 4/15
107/107 [==============================] - 27s 253ms/step - loss: 0.3529 - accuracy: 0.8481 - val_loss: 0.4232 - val_accuracy: 0.7937
Epoch 5/15
107/107 [==============================] - 28s 260ms/step - loss: 0.3378 - accuracy: 0.8437 - val_loss: 0.4363 - val_accuracy: 0.7646
Epoch 6/15
107/107 [==============================] - 24s 227ms/step - loss: 0.3072 - accuracy: 0.8638 - val_loss: 0.4630 - val_accuracy: 0.7963
Epoch 7/15
107/107 [==============================] - 25s 234ms/step - loss: 0.2218 - accuracy: 0.9086 - val_loss: 0.

Epoch 2/15
107/107 [==============================] - 69s 644ms/step - loss: 0.4826 - accuracy: 0.7628 - val_loss: 0.4335 - val_accuracy: 0.8037
Epoch 3/15
107/107 [==============================] - 61s 571ms/step - loss: 0.4091 - accuracy: 0.8145 - val_loss: 0.4195 - val_accuracy: 0.8090
Epoch 4/15
107/107 [==============================] - 61s 570ms/step - loss: 0.3551 - accuracy: 0.8455 - val_loss: 0.4173 - val_accuracy: 0.8064
Epoch 5/15
107/107 [==============================] - 61s 568ms/step - loss: 0.3220 - accuracy: 0.8607 - val_loss: 0.4444 - val_accuracy: 0.7905
Epoch 6/15
107/107 [==============================] - 60s 565ms/step - loss: 0.2442 - accuracy: 0.8947 - val_loss: 0.4714 - val_accuracy: 0.7878
Epoch 7/15
107/107 [==============================] - 60s 561ms/step - loss: 0.2019 - accuracy: 0.9163 - val_loss: 0.5042 - val_accuracy: 0.8037
Epoch 8/15
107/107 [==============================] - 60s 560ms/step - loss: 0.1791 - accuracy: 0.9330 - val_loss: 0.7453 - val_ac

## Summary

In [31]:
record2

acc1       acc2       acc3      acc4       acc5       acc6       acc7  \
0  79.62963  79.100531  76.455027  81.74603  77.248675  80.901855  80.636603   

        acc8       acc9      acc10        AVG  
0  79.310346  81.697613  77.984083  79.471039

In [32]:
report = record2
report = report.to_excel('LSTM_CR_v2_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## LSTM Model

In [33]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [34]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_48 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_49 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 129       
Total params: 585,825
Trainable params: 585,825
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [35]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [36]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
107/107 [==============================] - 145s 1s/step - loss: 0.6074 - accuracy: 0.6530 - val_loss: 0.5082 - val_accuracy: 0.7725
Epoch 2/15
107/107 [==============================] - 94s 884ms/step - loss: 0.3121 - accuracy: 0.8698 - val_loss: 0.4584 - val_accuracy: 0.7937
Epoch 3/15
107/107 [==============================] - 85s 797ms/step - loss: 0.1915 - accuracy: 0.9274 - val_loss: 0.6119 - val_accuracy: 0.7593
Epoch 4/15
107/107 [==============================] - 85s 792ms/step - loss: 0.0979 - accuracy: 0.9680 - val_loss: 0.7193 - val_accuracy: 0.7487
Epoch 5/15
107/107 [==============================] - 84s 788ms/step - loss: 0.0774 - accuracy: 0.9725 - val_loss: 0.8153 - val_accuracy: 0.7593
Epoch 6/15
107/107 [==============================] - 84s 784ms/step - loss: 0.0410 - accuracy: 0.9851 - val_loss: 0.9553 - val_accuracy: 0.7698
Epoch 7/15
107/107 [==============================] - 84s 786ms/step - loss: 0.0226 - accuracy: 0.9934 - val_loss: 1.21

Test Accuracy: 81.69761300086975
Training 8: 
Epoch 1/15
107/107 [==============================] - 213s 2s/step - loss: 0.6031 - accuracy: 0.6610 - val_loss: 0.4561 - val_accuracy: 0.7666
Epoch 2/15
107/107 [==============================] - 180s 2s/step - loss: 0.3103 - accuracy: 0.8733 - val_loss: 0.4520 - val_accuracy: 0.7798
Epoch 3/15
107/107 [==============================] - 148s 1s/step - loss: 0.1730 - accuracy: 0.9385 - val_loss: 0.4837 - val_accuracy: 0.8037
Epoch 4/15
107/107 [==============================] - 149s 1s/step - loss: 0.1093 - accuracy: 0.9641 - val_loss: 0.5766 - val_accuracy: 0.7798
Epoch 5/15
107/107 [==============================] - 141s 1s/step - loss: 0.0644 - accuracy: 0.9783 - val_loss: 1.0456 - val_accuracy: 0.7692
Epoch 6/15
107/107 [==============================] - 133s 1s/step - loss: 0.0282 - accuracy: 0.9896 - val_loss: 0.8313 - val_accuracy: 0.7905
Epoch 7/15
107/107 [==============================] - 132s 1s/step - loss: 0.0290 - accuracy: 0.

## Summary

In [37]:
record3

acc1       acc2       acc3       acc4       acc5       acc6  \
0  79.365081  79.365081  78.042328  83.333331  83.597887  82.228118   

        acc7      acc8       acc9      acc10        AVG  
0  81.697613  80.37135  78.514588  82.493371  80.900875

In [38]:
report = record3
report = report.to_excel('LSTM_CR_v2_3.xlsx', sheet_name='dynamic')